In [1]:
#| default_exp main

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
#| export
from fasthtml.common import *
from monsterui.all import *
from components import *
from utils import *

In [5]:
#| export
# Filesystem path - for Python to find files
BASE_DIR = Path.cwd()
if BASE_DIR.name == 'nbs': BASE_DIR = BASE_DIR.parent
STATIC_DIR = BASE_DIR / "static"

IN_NOTEBOOK = 'ipykernel' in sys.modules
css_path = STATIC_DIR / "css" / "custom.css"

if IN_NOTEBOOK: css_hdr = Style(css_path.read_text())
else: css_hdr = Link(rel='stylesheet', href='/static/css/custom.css', type='text/css')

css_content = (STATIC_DIR / "css" / "custom.css").read_text()

app, rt = fast_app(
    hdrs=(Theme.blue.headers(highlightjs=True), css_hdr)
)

In [6]:
#| export
@rt("/")
def index(category: list[str] = None):
    """Homepage listing all blog posts"""
    all_posts = load_all_posts()
    all_posts.sort(key=lambda p: p['date'], reverse=True)

    selected_cats = [c.lower() for c in category] if category else []
    posts = filter_posts(all_posts, selected_cats)
    all_tags = get_all_tags()
    enabled_tags = get_enabled_tags(all_posts, selected_cats, all_tags)

    return Layout(        
        # Hero section
        Div(
            H1("Ivan Dolgushev", cls="text-4xl font-bold mb-3 text-gray-900"),
            P("Software engineer, writer, and technology enthusiast.",
              cls="text-xl text-gray-600 mb-8"),
            cls="mb-12"
        ),
        
        # Filter Section
        FilterSection(all_tags, selected_cats, list(enabled_tags)),

        # Blog posts section
        Div(
            H2("Recent Posts", cls="text-2xl font-bold mb-6 text-gray-900"),
            Div(
                *[PostCard(post) for post in posts] if posts else [
                    P("No blog posts yet. Check back soon!", cls="text-gray-600 py-6")
                ],
                cls="divide-y divide-gray-200"
            ),
            cls="mb-12"
        ),
        title="Ivan Dolgushev - Personal Blog"
    )

@app.get("/post/{slug}")
def post_page(slug: str):
    """Individual blog post page"""
    post = load_post_by_slug(slug)
    if not post: return Layout(NotFoundContent(), title="404 - Not Found" ), 404
    return Layout(PostContent(post), title=f"{post['title']} - My Blog", wide=True)

@app.get("/about")
def about():
    """About page"""
    return Layout(AboutContent(), title="About - My Blog")

@app.get("/contact")
def contact():
    """Contact page"""
    return Layout(ContactForm(),title="Contact - My Blog")

@app.post("/contact/submit")
async def contact_submit(name: str, email: str, message: str):
    """Handle contact form submission"""
    # TODO: Send email or save to database
    return Layout(
        Div(
            H1("Thank You!", cls="text-4xl font-bold mb-4 text-center text-success"),
            P(f"Thanks for reaching out, {name}!", cls="text-center text-lg mb-2"),
            P("Your message has been received. I'll get back to you soon!", cls="text-center text-gray-600 mb-6"),
            Div(A(Button("Back to Home", cls=ButtonT.primary), href="/"), cls="text-center"),
            cls="py-12"
        ),
        title="Message Sent - My Blog"
    )

In [13]:
from fasthtml.jupyter import JupyUvi, HTMX
server = JupyUvi(app)

In [8]:
#|export
@rt("/debug")
def debug():
    """Debug route to check file paths"""
    from pathlib import Path
    import os
    
    cwd = Path.cwd()
    static_path = STATIC_DIR if 'STATIC_DIR' in dir() else Path("static")
    css_path = static_path / "css" / "custom.css"
    
    info = f"""
    Current working directory: {cwd}
    STATIC_DIR: {static_path}
    STATIC_DIR exists: {static_path.exists()}
    CSS path: {css_path}
    CSS exists: {css_path.exists()}
    Directory listing of {static_path}:
    {list(static_path.glob('**/*')) if static_path.exists() else 'N/A'}
    """
    return Pre(info)


In [ ]:
HTMX()

In [12]:
server.stop()

In [ ]:
#| export
serve(port=int(os.getenv('PORT', 5001)), host='0.0.0.0')

In [9]:
from nbdev.export import nb_export
nb_export('03_main.ipynb', '../')